In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import glob
import datetime


In [ ]:
# Load data
def load_data(data_path):
    if os.path.isfile(data_path):
        data = np.genfromtxt(data_path, names=True, delimiter=",")
        return data
    else:
        print("Could not open file", data_path)
        return None
    
def load_csv_files(directory, data_type, data_dict):
    csv_files = glob.glob(os.path.join(directory, '*' + data_type + '.csv'))

    if data_dict is None:
        data_dict = {}

    for csv_file in csv_files:
        label = os.path.splitext(os.path.basename(csv_file))[0]
        if label.startswith("_"):
            continue
        data = load_data(csv_file)
        label = label[:label.rfind(data_type)-1]
        data_dict[label] = data

    if len(data_dict) == 0:
        print("Failed to load any data!")

def filter_nan(data_dict):
    for label, data in data_dict.items():
        data_clean = data[~np.isnan(data["stability"])]
        count_cleaned = len(data["stability"]) - len(data_clean["stability"])
        if count_cleaned > 0:
            print(f"[WARNING] {label}: Removed {count_cleaned} entries with nan stability")
            data_dict[label] = data_clean
    
folders = [
    ""
    ]
data_dict_imu = {}
data_dict_stability = {}
for folder in folders:
    load_csv_files(folder, "imu", data_dict_imu)
    load_csv_files(folder, "stability", data_dict_stability)
filter_nan(data_dict_stability)
print(f"Loaded {len(data_dict_imu)} imu datasets.")
for label, data_imu in data_dict_imu.items():
    print(label)
print(f"Loaded {len(data_dict_stability)} stability datasets.")
for label, data_imu in data_dict_stability.items():
    print(label)

In [ ]:
def compute_binned_stability(travelled_distance, stability, bin_size):
    # Define bin edges
    min_distance = travelled_distance.min()
    max_distance = travelled_distance.max()
    bins = np.arange(min_distance, max_distance + bin_size, bin_size)
    # print(bins)

    # Assign each travelled_distance value to a bin
    bin_indices = np.digitize(travelled_distance, bins)

    # Calculate the average stability for each bin
    binned_stability = np.zeros(len(bins) - 1)
    for i in range(1, len(bins)):
        indices = np.where(bin_indices == i)[0]
        if indices.size > 0:
            binned_stability[i-1] = np.mean(stability[indices])
        else:
            print(f"[WARNING] Bin ({bins[i-1]},{bins[i]}) is empty")

    bin_values = bins[0:len(bins)-1] + bin_size / 2.0

    # The binned_stability dictionary now contains the average stability values for each bin
    # print(bin_values)
    # print(binned_stability)
    # for bin_range, avg_stability in binned_stability.items():
    #     print(f"Bin {bin_range}: Average Stability = {avg_stability}")

    return bin_values, binned_stability

def compute_moving_average_over_distance(travelled_distance, stability, window_distance=1.0):
    # Initialize the list to hold the running averages
    running_averages = np.empty_like(stability)

    # Compute running average with a 1m window
    for i in range(len(travelled_distance)):
        # Determine the start and end of the window
        start_distance = travelled_distance[i] - window_distance / 2
        end_distance = travelled_distance[i] + window_distance / 2
        
        # Find indices where distances fall within the window
        window_indices = np.where((travelled_distance >= start_distance) & (travelled_distance <= end_distance))[0]
        
        # Compute the average stability for these indices
        running_averages[i] = np.mean(stability[window_indices])
    return running_averages

def moving_average(data, size):
    window = np.ones(int(size)) / float(size)
    return np.convolve(data, window, 'same')




In [ ]:
# Stability plot
# Raw relative probability per sample
fig, ax = plt.subplots()
ax.title.set_text("Stability")
for label, data in data_dict_stability.items():

    # bin_size = 0.15
    # bin_values, binned_stability = compute_binned_stability(data["travelled_distance"], data["stability"], bin_size)
    # ax.bar(bin_values, binned_stability, width=bin_size)


    # ax.plot(data["travelled_distance"], data["stability"], label=label)
    
    # smoothed_stability = compute_moving_average_over_distance(data["travelled_distance"], data["stability"], window_distance=0.25)
    # ax.plot(data["travelled_distance"], smoothed_stability, label=label + "-smooth")
    ax.plot(data["travelled_distance"], data["stability"], label=label)

ax.legend()
plt.show()

In [ ]:
def compute_statistics(array, label: str):
    avg = np.mean(array)
    sum = np.sum(array)
    maximum = np.max(array)
    percentile = np.percentile(array, 99)
    # print(f" -- Avg. {label}: {avg:.2f}, Acc. {label}: {sum:.2f}, Max. {label}: {maximum:.2f}, Perc. {label}: {percentile:.2f}") 
    return avg, sum, maximum, percentile

def compute_integral(time, data):
    return np.trapz(data, time)

def compute_smoothness_metric(x, y, z, calibration, min, max):
    vec = np.stack((x, y, z), axis=0)
    norm = np.linalg.norm(vec, axis=0) - calibration
    norm = np.maximum(norm, min)
    norm = np.minimum(norm, max)
    return norm*norm 

def compute_shock(acc_x, acc_y, acc_z):
    calibration = 10.0 # only consider hard hits
    gravity = 9.81
    return compute_smoothness_metric(acc_x, acc_y, acc_z, calibration + gravity, 0, 50.0)

def compute_swing(vel_x, vel_y):
    calibration = 0.5 # only consider fast swings
    return compute_smoothness_metric(vel_x, vel_y, np.zeros_like(vel_x), calibration, 0, 3.0)

# Compute trial metrics
def compute_metrics(data_imu, data_stability):
    # time
    duration = max(data_imu["time"][-1], data_stability["time"][-1])
    # print(f" -- Duration: {duration:.2f}s")

    # Average stability
    avg_stability = np.mean(data_stability["stability"])
    # _, binned_stability = compute_binned_stability(data_stability["travelled_distance"], data_stability["stability"], 0.12)
    # avg_binned_stability = np.mean(binned_stability)
    # smoothed_stability = compute_moving_average_over_distance(data_stability["travelled_distance"], data_stability["stability"], window_distance=1.0)
    # smoothed_stability = moving_average(data_stability["stability"], size=5)
    # avg_smoothed_stability = np.mean(smoothed_stability)
    # print(f" -- Avg. stability: {avg_stability:.2f}, avg. binned stability: {avg_binned_stability:.2f}, avg. smoothed stability: {avg_smoothed_stability:.2f}")

    # Locomotion roughness
    shock = compute_shock(data_imu["linear_acceleration_x"], data_imu["linear_acceleration_y"], data_imu["linear_acceleration_z"])
    # avg_shock, sum_shock, max_shock, percentile_shock = compute_statistics(shock, "shock")
    int_shock = compute_integral(data_imu["time"], shock)

    swing = compute_swing(data_imu["angular_velocity_x"], data_imu["angular_velocity_y"])
    # avg_swing, sum_swing, max_swing, percentile_swing = compute_statistics(swing, "swing")
    int_swing = compute_integral(data_imu["time"], swing)

    return duration, avg_stability, int_shock, int_swing

def print_results(result_data):
    for label, trials in result_data.items():
        print("Mode:", label)
        print("Duration Stability Shock Swing")
        for i, trial in enumerate(trials):
            metrics_str = ', '.join(str(np.around(num, decimals=2)) for num in trial)
            print(metrics_str)
        trials_matrix = np.array(trials)
        avg = np.mean(trials_matrix, axis=0)
        print(f"Avg:     {', '.join(str(np.around(num, decimals=2)) for num in avg)}")

        print("")


result_data = dict()
for label, data_imu in data_dict_imu.items():
    # print(f"Trial {label} metrics:")
    data_stability = data_dict_stability[label]
    metrics = compute_metrics(data_imu, data_stability)
    mode = label[:label.rfind("trial") -1]
    if mode in result_data:
        result_data[mode].append(metrics)
    else:
        result_data[mode] = []
        result_data[mode].append(metrics)

print_results(result_data)

In [ ]:
fig, axs = plt.subplots(2, 1)
axs[0].title.set_text("Shock")
axs[1].title.set_text("Swing")
for label, data in data_dict_imu.items():
    shock = compute_shock(data["linear_acceleration_x"], data["linear_acceleration_y"], data["linear_acceleration_z"])
    axs[0].plot(data["time"], shock, label=label)

    swing = compute_swing(data["angular_velocity_x"], data["angular_velocity_y"])
    swing = np.maximum(swing, 0.0)

    axs[1].plot(data["time"], swing, label=label)
    # axs[1].plot(data["time"], data["angular_velocity_z"])

for ax in axs:
    ax.legend()
    ax.grid()


In [ ]:
# Per mode: scatter plot of shock/swing over time

# Collect data per mode
modes = ["autonomy", "teleop", "wb_assistance",]
mode_labels = {
    "autonomy": "Supervised Autonomy",
    "teleop": "Guided + Assisted Teleoperation",
    "wb_assistance": "Shared Autonomy",
}
mode_results_dict = dict()
for label, trials in result_data.items():
    trials_matrix = np.array(trials)
    for mode in modes:
        if mode in label:
            if mode in mode_results_dict:
                stacked = np.concatenate((mode_results_dict[mode], trials_matrix), axis=0)
                mode_results_dict[mode] = stacked
            else:
                mode_results_dict[mode] = trials_matrix
# print(mode_results_dict["wb_assistance"][:, 0].min(), mode_results_dict["wb_assistance"][:, 0].max())


# Plot
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
bins = np.array([150, 200, 250, 300, 350])
# bins = np.arange(150, 400, 60)
# bins = np.array([mode_results_dict["wb_assistance"][:, 0].min(), mode_results_dict["wb_assistance"][:, 0].max()])
# bins = np.arange(0, 500, 25)
print("Bins", bins)
for i, (mode, results) in enumerate(mode_results_dict.items()):
    axs[0].set_title("Shock")
    axs[0].scatter(results[:, 0], results[:, 2], label=mode_labels[mode])
    axs[0].set_ylabel("Shock")

    axs[1].set_title("Swing")
    axs[1].scatter(results[:, 0], results[:, 2], label=mode_labels[mode])
    axs[1].set_ylabel("Swing")

for ax in axs:
    ax.legend()

In [ ]:
def plot_avg_histogram(ax, duration, metric, bins, bar_index, total_bars, label):
    indices = np.digitize(duration, bins) - 1  # Find which bin each time value falls into
    n_bins = len(bins) -1
    # Compute average shock values for each bin
    avg_shocks = np.array([metric[indices == i].mean() if len(metric[indices == i]) > 0 else 0 for i in range(n_bins)])
    std_dev = np.array([metric[indices == i].std() if len(metric[indices == i]) > 0 else 0 for i in range(n_bins)])

    # Plot
    x = np.arange(n_bins)
    default_bar_width = 1.0
    bar_width = default_bar_width / total_bars
    ax.bar(x + bar_index * bar_width, avg_shocks, width=bar_width, yerr=std_dev, edgecolor='black', align='edge', label=label)

    ax.set_xticks(x , labels=[f"{bins[i]:.2f}" for i in range(n_bins)])
    ax.set_xlabel('Duration (s)')
    ax.set_xlim(0, n_bins)
    # ax.set_ylim(0, avg_shocks.max() * 1.5)
    # ax.set_title('Average Shock per Time Bin')


# Plot
fig2, axs2 = plt.subplots(1, 2, figsize=(10, 4))
fig2.tight_layout()
bins = np.array([150, 200, 250, 300, 350])
# bins = np.arange(150, 400, 60)
# bins = np.array([mode_results_dict["wb_assistance"][:, 0].min(), mode_results_dict["wb_assistance"][:, 0].max()])
# bins = np.arange(0, 500, 25)
print("Bins", bins)
for i, mode in enumerate(modes):
    results = mode_results_dict[mode]
    axs2[0].set_title("Shock")
    plot_avg_histogram(axs2[0], results[:, 0], results[:, 2], bins, i, len(mode_results_dict), label=mode_labels[mode])
    axs2[0].set_ylabel("Shock")

    axs2[1].set_title("Swing")
    plot_avg_histogram(axs2[1], results[:, 0], results[:, 3], bins, i, len(mode_results_dict), label=mode_labels[mode])
    axs2[1].set_ylabel("Swing")

for ax in axs2:
    ax.legend()

plt.savefig("export/locomotion_smoothness_over_time", dpi=300, bbox_inches = "tight")





In [ ]:
# plot with average time over trials per mode
# x-axis: trial #
# y-axis: avg duration per trial #
# 4 lines + scatter for each mode

# collect data
modes = ["autonomy", "guided_teleop", "teleop", "wb_assistance",]
mode_labels = {
    "autonomy": "Supervised Autonomy",
    "teleop": "Assisted Teleoperation",
    "guided_teleop": "Guided Teleoperation",
    "wb_assistance": "Shared Autonomy",
}
trail_comp_dict = dict()
for label, trials in result_data.items():
    trials_matrix = np.array(trials)
    for mode in modes:
        if mode in label:
            trial_durations = trials_matrix[:, 0:1]
            if mode in trail_comp_dict:
                concat = np.concatenate((trail_comp_dict[mode], trial_durations), axis=1)
                trail_comp_dict[mode] = concat
            else:
                trail_comp_dict[mode] = trial_durations
            break


# plot
fig, ax = plt.subplots()
fig.tight_layout()
markers = ["o", "x", "D", "s"]
max_trials = 0
for _, results in trail_comp_dict.items():
    trials_num = results.shape[0]
    if trials_num > max_trials:
        max_trials = trials_num
x = np.arange(1, max_trials + 1, 1)
for i, ((mode, results), marker) in enumerate(zip(trail_comp_dict.items(), markers)):
    if results.shape[1] > 1:
        y = np.mean(results, axis=1)
        y_err = np.std(results, axis=1)
    else:
        y = results.transpose()[0]
        y_err = np.zeros_like(y)
    offset = 0.04
    x_offset = x + offset * (i - len(modes) / 2.0)
    ax.errorbar(x_offset, y, yerr=y_err, label=mode_labels[mode], fmt='--'+marker, ecolor="black")

ax.legend()
# ax.set_title("")
ax.set_xticks(x)
ax.set_xlabel("Trial #")
ax.set_ylabel("Avg. Duration (s)")
# ax.set_ylim(140, 500)
plt.savefig("export/trial_comparison", dpi=300, bbox_inches = "tight")

In [ ]:
# operator experience comparison
# plot x-axis: modes, 1 bar per operator per mode
# y-axis duration

# collect data
modes = ["teleop", "wb_assistance",]
mode_labels = {
    "autonomy": "Supervised Autonomy",
    "teleop": "Assisted + Guided Teleoperation",
    "guided_teleop": "Guided Teleoperation",
    "wb_assistance": "Shared Autonomy",
}

operator_experience = ["beginner", "intermediate", "expert"]
operator_experience_labels = {
    "beginner": "Beginner",
    "intermediate": "Intermediate",
    "expert": "Expert",
}

operator_comp_dict = dict()
for exp in operator_experience:
    operator_comp_dict[exp] = np.zeros((3, len(modes)))

for label, trials in result_data.items():
    trials_matrix = np.array(trials)
    for exp in operator_experience:
        if exp in label:
            trial_durations = trials_matrix[:, 0:1]
            # find mode index
            for i, mode in enumerate(modes):
                if mode in label:
                    mode_index = i
                    break
                if i == len(modes) - 1:
                    print("[WARN] Could not find mode in", label)

            operator_comp_dict[exp][:, mode_index:mode_index+1] = trial_durations[:]
            break

# plot
fig, ax = plt.subplots()
fig.tight_layout()

x = np.arange(0, len(modes), 1)
default_bar_width = 0.8
bar_width = default_bar_width / len(operator_experience)
for i, (op_exp, results) in enumerate(operator_comp_dict.items()):
    y = np.mean(results, axis=0)
    # print(op_exp, y)
    y_err = np.std(results, axis=0)
    x_offset = x + (i - len(operator_experience) / 2.0) * bar_width
    ax.bar(x_offset, y, width=bar_width, yerr=y_err, edgecolor='black', align='edge', label=operator_experience_labels[op_exp])

ax.legend()
ax.set_xticks(x , labels=[mode_labels[modes[i]] for i in range(len(modes))])
ax.set_ylabel("Avg. Duration (s)")

plt.savefig("export/operator_comparison", dpi=300, bbox_inches = "tight")
plt.show()

In [ ]:
def relative_to_absolute_time(time_array, reference_time_str):
    reference_stamp = datetime.datetime.strptime(reference_time_str, '%Y-%m-%d-%H:%M:%S.%f').replace(tzinfo=datetime.timezone.utc)
    absolute_stamps = np.array([reference_stamp + datetime.timedelta(seconds=int(s)) for s in time_array])
    return absolute_stamps

def absolute_to_relative_time(time_array, reference_datetime):
    relative_stamps = np.array([(absolute_time - reference_datetime).total_seconds() for absolute_time in time_array])
    return relative_stamps

def filter_data(time_stamps, data, start_time, end_time):
    filter_indices_start = time_stamps >= start_time
    filtered_time = time_stamps[filter_indices_start]
    filtered_data = [d[filter_indices_start] for d in data]

    filter_indices_end = filtered_time < end_time
    filtered_time = filtered_time[filter_indices_end]
    filtered_data = [d[filter_indices_end] for d in filtered_data]


    return filtered_time, filtered_data


# Plan vs execution comparison plot
data_stability_plan = load_data("")
plan_stamp_str = "2024-03-22-16:01:50.233347"
plan_stamps = relative_to_absolute_time(data_stability_plan["time"], plan_stamp_str)
plan_stamps = plan_stamps[1:]  # remove first invalid element
plan_stability = data_stability_plan["stability"][1:]
plan_robot_x = data_stability_plan["result_position_x"][1:]

data_stability_measured = data_dict_stability["autonomy_2024-03-22-16-58-16_trial-0"]
trial_stamp_str = "2024-03-22-16:01:06.552530"
trial_stamps = relative_to_absolute_time(data_stability_measured["time"], trial_stamp_str)
trial_stability = data_stability_measured["stability"]
trial_robot_x = data_stability_measured["robot_pose_position_x"]

# Smooth data
plan_stability = moving_average(plan_stability, 1)
trial_stability = moving_average(trial_stability, 5)

# Cut data to overlapping duration
start_time = plan_stamps[0] if plan_stamps[0] >= trial_stamps[0] else trial_stamps[0]
end_time = plan_stamps[-1] if plan_stamps[-1] < trial_stamps[-1] else trial_stamps[-1]

plan_stamps, [plan_stability, plan_robot_x] = filter_data(plan_stamps, [plan_stability, plan_robot_x], start_time, end_time)
trial_stamps, [trial_stability, trial_robot_x] = filter_data(trial_stamps, [trial_stability, trial_robot_x], start_time, end_time)

# Convert to relative start time
plan_stamps = absolute_to_relative_time(plan_stamps, start_time)
trial_stamps = absolute_to_relative_time(trial_stamps, start_time)

# print("Plan points/second:", len(plan_stamps)/plan_stamps[-1])
# print("Trial points/second:", len(trial_stamps)/trial_stamps[-1])


# Stability plot
fig, ax = plt.subplots(1, 1, dpi=200, figsize=(10, 5))
fig.tight_layout()

# axs[0].plot(plan_stamps, plan_stability, label="Plan")
# axs[0].plot(trial_stamps, trial_stability, label="Measured")
# axs[0].set_xlabel("Time (s)")

ax.plot(plan_robot_x, plan_stability, label="Plan", linestyle="dotted")
ax.plot(trial_robot_x, trial_stability, label="Estimation")
# axs[1].plot(trial_robot_x, trial_stability_smooth, label="Measured-smooth")

ax.set_xlabel("Position (m)")
ax.set_ylabel("Stability Margin (FASM)")
ax.legend()
ax.set_ylim(0, 4)
ax.set_xticks(np.arange(-0.5, 5, 0.5))

plt.savefig("export/plan_comparison_stability", dpi=300, bbox_inches = "tight")
plt.show()
